In [ ]:
!pip install --upgrade gensim

     |████████████████████████████████| 24.1 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file1 = open('/content/drive/MyDrive/src_file.txt', 'r')
source = file1.readlines()
file1 = open('/content/drive/MyDrive/685 Project/baseline_predict.txt', 'r')
predict = file1.readlines()


In [ ]:
import re
from gensim.utils import simple_preprocess

# From: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/soft_cosine_tutorial.ipynb
def preprocess(doc):
    # Tokenize, clean up input document string
    doc = re.sub('[^A-Za-z0-9]+',' ',doc).strip()
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf"))]

# Preprocess the documents, including the query string
predict = [preprocess(p) for p in predict]
source = [preprocess(s) for s in source]

In [ ]:
import gensim.downloader as api
import numpy as np
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.similarities import SoftCosineSimilarity

# Load the model: this is a big file, can take a while to download and open
glove = api.load("glove-wiki-gigaword-50")    
similarity_index = WordEmbeddingSimilarityIndex(glove)

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
sim_score = []
for pair in range(len(predict)):
  # Build the term dictionary, TF-idf model
  dictionary = Dictionary([source[pair],predict[pair]])
  tfidf = TfidfModel(dictionary=dictionary)
  # Create the term similarity matrix.  
  similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf)
  query_tf = tfidf[dictionary.doc2bow(predict[pair])]

  index = SoftCosineSimilarity(
              tfidf[dictionary.doc2bow(source[pair])],
              similarity_matrix)

  doc_similarity_score = index[query_tf]
  #print(doc_similarity_score.item())
  # Output the sorted similarity scores and documents
  sim_score.append(doc_similarity_score.item())

print(np.average(sim_score))

100%|██████████| 41/41 [00:00<00:00, 43.89it/s]

0.35847748421570835
